In [1]:
%reset
import tensorflow as tf
import keras
import pickle as pk
import matplotlib.pyplot as pl
import matplotlib
import numpy as np
import pdb

%matplotlib inline

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


2021-10-28 00:34:44.350096: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


# The neural network architecture that we'll use in the example below is based on:

See https://github.com/eriklindernoren/Keras-GAN/blob/master/context_encoder/context_encoder.py

## Data location and dimensions

In [2]:
#The data are "cutouts" from full-sky maps made from observations by the Planck satellite.
#These maps were constructed at an observing frequency of 857 GHz
data_dir = './cutout_data/'
# this file contains some cutouts that are "off star", i.e. centered at random locations on the sky
# and not on star locations
offstar_file = 'cutouts_1000offstarfirstpass_01.pk'
offstar_multi = 'cutouts_1000offstarmultifreq_01.pk'
#The iniital cutouts have a dimension of 200x200.  We'll reduce the dimension to dim_reduced x dim_reduced
#to speed up the analysis
dim_reduced = 32
#We're going to mask out a small part of the center of the cutout with dimension dim_mask x dim_mask
dim_mask = 8

## Load the cutout data

In [3]:
#load the data
#cutout_data = pk.load(open(data_dir + offstar_file, 'rb'))
#print(cutout_data.keys())
#ll = cutout_data['l'] # galatic longitude of the cutout
#bb = cutout_data['b'] # galactic latitude of the cutout
#cutouts_orig = cutout_data['cutouts']
#print(cutouts_orig.shape)

In [4]:
#multi freq
cutout_data_mf = pk.load(open(data_dir + offstar_multi, 'rb'))
print(cutout_data_mf.keys())
ll = cutout_data_mf['l'] # galatic longitude of the cutout
bb = cutout_data_mf['b'] # galactic latitude of the cutout

cutouts_orig_mf = cutout_data_mf['cutouts']
print(cutout_data_mf['planck_files'])
print(cutouts_orig_mf.shape)
#one-axis array
cutouts_multi = np.append(cutouts_orig_mf[0], cutouts_orig_mf[1], axis=0)
print(cutouts_multi.shape)

dict_keys(['planck_files', 'cutouts', 'l', 'b'])
['planck_545.fits', 'planck_857.fits']
(2, 1000, 200, 200)
(2000, 200, 200)


## Plot a few cutouts

In [5]:
nplots = 6
fig, ax = pl.subplots(2,nplots, figsize = (nplots*3., 4))
for ii in range(0,nplots):
    ax[0,ii].imshow(np.log(cutouts_multi[ii,:,:]))
    ax[1,ii].imshow(np.log(cutouts_multi[1000 + ii,:,:]))
    ax[0,ii].yaxis.set_visible(False)
    ax[1,ii].yaxis.set_visible(False)

## Reduce the dimensionality of the cutouts
And generate more data by flipping images

(8000, 32, 32)


In [6]:
orig_dims = cutouts_multi.shape
min_ind = int(orig_dims[1]/2 - dim_reduced/2)
max_ind = int(orig_dims[1]/2 + dim_reduced/2)
cutouts = cutouts_multi[:, min_ind:max_ind, min_ind:max_ind]

#flip horizontaly and append
cutouts_fliplr = np.flip(cutouts,axis=2)
cutouts = np.append(cutouts,cutouts_fliplr,axis=0)

#flip vert and app
cutouts_flipud = np.flip(cutouts,axis=1)
cutouts = np.append(cutouts,cutouts_flipud,axis=0)

print(cutouts.shape)

#check by plot
fig, ax = pl.subplots(2,2, figsize = (nplots*3., 4))
ax[0,0].imshow(np.log(cutouts[0,:,:]))
ax[0,1].imshow(np.log(cutouts[2000,:,:]))

ax[1,0].imshow(np.log(cutouts[4000,:,:]))
ax[1,1].imshow(np.log(cutouts[6000,:,:]))
    
    
#fig.savefig('./network_out/example_images.png')

## Set up the neural network

In [8]:
#train test split
from sklearn.model_selection import train_test_split


In [13]:
from __future__ import print_function, division

from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras import losses
from tensorflow.keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt

import numpy as np

class ContextEncoder():
    def __init__(self):
        self.img_rows = dim_reduced
        self.img_cols = dim_reduced
        self.mask_height = dim_mask
        self.mask_width = dim_mask
        #777777777777
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.missing_shape = (self.mask_height, self.mask_width, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates the missing
        # part of the image
        masked_img = Input(shape=self.img_shape)
        gen_missing = self.generator(masked_img)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines
        # if it is generated or if it is a real image
        valid = self.discriminator(gen_missing)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(masked_img , [gen_missing, valid])
        self.combined.compile(loss=['mse', 'binary_crossentropy'],
            loss_weights = l_weights,
            #loss_weights=[0.999, 0.001],
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        # Encoder
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2D(512, kernel_size=1, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))

        # Decoder
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation('relu'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation('relu'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation('tanh'))

        model.summary()

        masked_img = Input(shape=self.img_shape)
        gen_missing = model(masked_img)

        return Model(masked_img, gen_missing)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=self.missing_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.missing_shape)
        validity = model(img)

        return Model(img, validity)

    def mask_randomly(self, imgs):
        y1 = np.random.randint(0, self.img_rows - self.mask_height, imgs.shape[0])
        y2 = y1 + self.mask_height
        x1 = np.random.randint(0, self.img_rows - self.mask_width, imgs.shape[0])
        x2 = x1 + self.mask_width

        masked_imgs = np.empty_like(imgs)
        missing_parts = np.empty((imgs.shape[0], self.mask_height, self.mask_width, self.channels))
        for i, img in enumerate(imgs):
            masked_img = img.copy()
            _y1, _y2, _x1, _x2 = y1[i], y2[i], x1[i], x2[i]
            missing_parts[i] = masked_img[_y1:_y2, _x1:_x2, :].copy()
            masked_img[_y1:_y2, _x1:_x2, :] = 0
            masked_imgs[i] = masked_img

        return masked_imgs, missing_parts, (y1, y2, x1, x2)

    def mask_center(self, imgs):
        y1 = int(self.img_rows/2) - int(self.mask_height/2)
        y2 = y1 + self.mask_height
        x1 = int(self.img_rows/2) - int(self.mask_width/2)
        x2 = x1 + self.mask_width

        masked_imgs = np.empty_like(imgs)
        missing_parts = np.empty((imgs.shape[0], self.mask_height, self.mask_width, self.channels))
        for i, img in enumerate(imgs):
            masked_img = img.copy()
            missing_parts[i] = masked_img[y1:y2, x1:x2, :].copy()
            masked_img[y1:y2, x1:x2, :] = -100
            masked_imgs[i] = masked_img

        return masked_imgs, missing_parts, (y1, y2, x1, x2)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        #(X_train, y_train), (_, _) = cifar10.load_data()
        #pdb.set_trace()
        X_train = np.expand_dims(np.log(cutouts),axis=3)
        
        
        # Rescale -1 to 1
        #77777777777
        X_mean = np.mean(X_train)
        X_std = np.std(X_train)
        X_train = (X_train - X_mean)/X_std
        #y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        d_loss_out = []
        g_loss_out = []

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            masked_imgs, missing_parts, _ = self.mask_center(imgs)

            # Generate a batch of new images
            gen_missing = self.generator.predict(masked_imgs)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(missing_parts, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_missing, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            d_loss_out.append(d_loss[0])

            # ---------------------
            #  Train Generator
            # ---------------------

            g_loss = self.combined.train_on_batch(masked_imgs, [missing_parts, valid])
            g_loss_out.append(g_loss[0])

            # Plot the progress
            #print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f, mse: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], g_loss[1]))
            
            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                idx = np.random.randint(0, X_train.shape[0], 6)
                imgs = X_train[idx]
                self.sample_images(epoch, imgs)
        self.d_loss_plot = d_loss_out
        self.g_loss_plot = g_loss_out

    def sample_images(self, epoch, imgs):
        r, c = 3, 6

        masked_imgs, missing_parts, (y1, y2, x1, x2) = self.mask_center(imgs)
        gen_missing = self.generator.predict(masked_imgs)

        imgs = 0.5 * imgs + 0.5
        masked_imgs = 0.5 * masked_imgs + 0.5
        gen_missing = 0.5 * gen_missing + 0.5

        fig, axs = plt.subplots(r, c)
        for i in range(c):
            axs[0,i].imshow(imgs[i, :,:], vmin = 0., vmax = 1.0)
            axs[0,i].axis('off')
            axs[1,i].imshow(masked_imgs[i, :,:], vmin = 0., vmax = 1.0)
            axs[1,i].axis('off')
            filled_in = imgs[i].copy()
            filled_in[y1:y2, x1:x2, :] = gen_missing[i]
            axs[2,i].imshow(filled_in, vmin = 0., vmax = 1.0)
            axs[2,i].axis('off')
        img_name = f"{epoch},{l_weights}"
        #fig.savefig("network_output/images/%d.png" % epoch)
        fig.savefig(f"network_output/images/{img_name}.png")
        plt.close()
        
    def save_model(self):

        def save(model, model_name):
            model_path = "network_output/saved_model/%s.json" % model_name
            weights_path = "network_output/saved_model/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "generator")
        save(self.discriminator, "discriminator")




In [14]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [15]:
l_weights= [0.999, 0.001]
epochs = 201
cne = ContextEncoder()
cne.train(epochs=epochs, batch_size=64, sample_interval=100)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 4, 4, 64)          640       
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 4, 4, 64)          0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 4, 4, 64)          256       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 2, 2, 128)         73856     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 2, 2, 128)         0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 2, 2, 128)         512       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 2, 2, 256)        

<class 'list'>


In [23]:
#graph loss functionvs epoch 
print(type(cne.g_loss_plot))
total_loss = 0.999*np.array(cne.g_loss_plot) + 0.001*np.array(cne.d_loss_plot)
epochs_plot = np.arange(1,epochs+1,1)

plt.figure(figsize=(15,5))
plt.plot(epochs_plot,total_loss,label="Total Loss",c='r')
#plt.plot(epochs_plot,cne.d_loss_plot,label="Discriminator Loss",c='b')
#plt.plot(epochs_plot,cne.g_loss_plot,label="Generator Loss",c='g')
plt.xlabel("Epochs")
plt.ylabel('Loss')
plt.show()

In [ ]:
#implement test train split


#invent test error, new test function without trianing